
# Cruise Control Module
### DARPA Assured Micro-Patching Project
### Peter Lobato, Matt Campo, Ben Ettlinger, Subhojeet Mukherjee




This jupyter notebook contains a simulated cruise control module (CCM). It pairs with a separate notebook, "VehicleModel," to be run on a separate kernel, which simulates the response of a vehicle.

The CCM takes in vehicle speed, via a CAN bus, and a desired speed setpoint and outputs accelerator pedal position (APP) via CAN bus. This runs continuously simulating a vehicle driving down a road using cruise control in real-time. 

The cruise control module settings can be configured by the user. These include the Kp and Ki terms, (+) and (-) slew rates, speed differential at which the CCM shuts off and integrator time constant.


## Requirements

The following are requirements for the cruise control module. Note that the vehicle model also has requirements which are listed in the Vehicle Model notebook.

### User Inputs

All user inputs are listed in the cell with the heading "#User-defined variables for CCM." The only exception is defining CAN bus parameters.

target: set vehicle speed to be maintained [kph]

c_kp: proportional term constant

c_ki: constant integral term constant

itc: integral time constant [used per time interval: units are number of time intervals, or 1/10th seconds]

slew_lowLimit: Slew rate limit for decreasing APP [%/s]

slew_highLimit: Slew rate limit for increasing APP [%/s]



### Vehicle Speed

Vehicle speed is read from bus "CAN1" using the J1939 standard.

- Program will look for this message once every 100ms (meant for 10Hz data rate).
- arbitration ID 0x18FEF125
- parameter group number (PGN) 65265
- 2-byte message on bytes 6 and 7 (bytes 5 and 6 if counting from 0).
- multiplier: 1/256
- offset: 0
- units: [kph]



   
### Accelerator Pedal Position (APP)

Accelerator pedal position is transmitted on bus "CAN1" using the J1939 standard.

- Message is sent every 100ms (meant for 10Hz data rate)
- arbitration ID 0x18F00326
- parameter group number (PGN) 61444
- 1-byte message on byte 7 (2nd from last)
- multiplier: 1
- offset: -125
- units: [%]

Complete J1939 standard is available through SAE. These values taken from J1939 Digital Annex.


In [1]:
# Import libraries
import time
import math
import numpy as np
import socket
from socket import timeout
import struct
import sys

In [2]:
# User-defined variables for CCM
target = 75.0  # Speed setpoint [kph]
c_kp = 20  # Constant proportional term
c_ki = 0.02  # Constant integral term
itc = 100  # Integral time constant [used per time interval, units are number of time intervals, or 1/10th seconds]
slew_lowLimit = -75  # Low limit for calculated slew rate [%/s]
slew_highLimit = 75  # High limit for calculated slew rate [%/s]

# These variables initialize the CCM and should not be changed
APP = 0  # Initial accelerator pedal position [%]
prev_APP = 0
prev_errors = [0.0 for _ in range(itc)]  # Array of previous errors to calculate integral term
i = 0  # Keeps circular buffer of size itc for error summation
checks = True  # Breaks proccess if errors occur
t_interval = .1  # Constant time interval [reflects CAN protocol wanted]

# The following are not needed if running on a Beaglebone.
# These are used if running notebook on a PC that supports matplotlib and can graph the response
# Using to check results via graph

#graph = True
#x = [0.0]
#y = [0.0]
#x_value = 0.0
#y_value = 0.0

## Vehicle Speed Receive over CAN

The following cell is a Python function which reads vehicle speed from CAN1 and passes its value to the main loop of this notebook. Loop timing happens in this function: a CAN socket opens and listens for the correct message for up to 100ms. If the message is received, the function waits the remaining time within the 100ms window before continuing. If the message is not received, the function breaks, holds the previous speed value, and continues the main response loop. Therefore, the main response loop always runs at 10Hz. This function has no argument input and passes out vehicle speed as a floating-point integer.

More information on the Python socketcan can be found here:

https://python-can.readthedocs.io/en/master/interfaces/socketcan.html

In [3]:
#This cell simulates the cruise control module receiving speed from the CAN bus
def CAN_kph_Rx():
    """Documentation: This function reads a single CAN message if it has ArbID 0x18FEF125 for vehicle speed."""
    
    global kph
    
    speed_m = 1/256 #Multiplier for speed from J1939 standard [counts/kph]
    speed_b = 0 #Offset for speed from J1939 [kph]
    
    # Open a socket and bind to it from SocketCAN
    sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
    sock.setsockopt(socket.SOL_CAN_RAW, socket.CAN_RAW_ERR_FILTER, socket.CAN_ERR_MASK) #allows socket to receive error frames
    # The interface can be seen from the command prompt (ifconfig)
    # The can channel must be configured using the ip link commands
    interface = "can1"

    # Bind to the interface
    try:
        sock.bind((interface,))
    except OSError:
        print("Could not bind to interface '%s'\n" % interface)
        sys.exit() 

    sock.settimeout(0.1) #testing this out
    # To match this data structure, the following struct format can be used:
    can_frame_format = "<LB3x8s"
    # Unsigned Long Integer (little endian), unsigned char (byte), three pad bytes, eight chars (bytes)
    # Note: this is 16 bytes


    # These are defined in can.h of the Linux kernel sources
    # e.g. https://github.com/torvalds/linux/blob/master/include/uapi/linux/can.h
    #print("socket.CAN_EFF_MASK = 0x{:08X}".format(socket.CAN_EFF_MASK))
    #print("socket.CAN_EFF_FLAG = 0x{:08X}".format(socket.CAN_EFF_FLAG))
    #print("socket.CAN_RTR_FLAG = 0x{:08X}".format(socket.CAN_RTR_FLAG))
    #print("socket.CAN_ERR_FLAG = 0x{:08X}".format(socket.CAN_ERR_FLAG))

    # Enter a loop to read and display the data
    t1, t2 = 0, 0
    
    while True:
        t1 = time.perf_counter()
        try:
            can_packet = sock.recv(16) 
        except socket.timeout:
            return
        can_id, can_dlc, can_data = struct.unpack(can_frame_format, can_packet)

        if (can_id == 2566844709): #ArbID for Speed. Change for APP
            break

    extended_frame = bool(can_id & socket.CAN_EFF_FLAG)
    error_frame = bool(can_id & socket.CAN_ERR_FLAG)
    remote_tx_req_frame = bool(can_id & socket.CAN_RTR_FLAG)

    if error_frame:
        can_id &= socket.CAN_ERR_MASK
        can_id_string = "{:08X} (ERROR)".format(can_id)
    else: #Data Frame
        if extended_frame:
            can_id &= socket.CAN_EFF_MASK
            can_id_string = "{:08X}".format(can_id)
        else: #Standard Frame
            can_id &= socket.CAN_SFF_MASK
            can_id_string = "{:03X}".format(can_id)

    if remote_tx_req_frame:
        can_id_string = "{:08X} (RTR)".format(can_id)

    hex_data_print = ' '.join(["{:02X}".format(can_byte) for can_byte in can_data])
    #print("{:12.6f} {} [{}] {}".format(time.time(), can_id_string, can_dlc, hex_data_print))
    kph = int(hex_data_print[18:20] + hex_data_print[15:17],base=16)*speed_m + speed_b

    t2 = time.perf_counter()
    if (t_interval - (t2-t1)) > 0:
        time.sleep(t_interval - (t2-t1))
    
    return kph #Units [kph]


## APP send over CAN

The following cell is a Python function which takes in the value of APP, as calculated in the Cruise Control Module function, and transmits it to CAN1. APP is taken in as the argument of the function, and the function has no return value.

In [4]:
#This cell simulates the cruise control module transmitting APP onto the CAN bus
def CAN_APP_Tx(APP):
    """Documentation: This function takes in the value 'Driver's Demand Engine - Percent Torque' as a
    floating-point number (type='float'), turns it into the J1939 CAN message format, and sends the message
    on bus 'VCAN0' using SocketCAN. It uses PGN 61444, SPN 512. Arbitration is set to 18. These can be changed"""

    APP_m = 1 #Multiplier for APP from J1939 standard [counts/%]
    APP_b = -125 #Offset for APP from J1939 [%]

    app_counts = int((APP - APP_b) / APP_m) #turns floating-point value from Python into base 10 integer number
    app_hex = hex(app_counts)
    app_byte1 = int(app_hex[2] + app_hex[3],16) #turns into a base-16 integer number
    #app_byte2 = int(app_hex[4] + app_hex[5],16)

    #print("APP: ",APP)
    #print("app_counts: ",app_counts)
    #print("app_hex: ",app_hex)
    #print("app_byte1: ",app_byte1)
    #print("app_byte2: ",app_byte2)
    
    # Open a socket and bind to it from SocketCAN
    sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)

    #Change this interface to match your desired connection
    interface = "can1"

    # Bind to the interface
    sock.bind((interface,))

    can_frame_format = "=lB3x8s"

    # Define the CAN Arbitration ID
    can_id = 0x18F00326 #For Driver's Demand on PGN 61444. Source Address = 26 (for no real reason)
    #Set the extended frame format bit.
    can_id |= socket.CAN_EFF_FLAG
    
    can_data = struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",0xFF) + struct.pack("B",app_byte1) + struct.pack("B",0xFF)
    #The data length code cannot be larger than 8
    can_dlc = min(len(can_data),8)
    
    #Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
    #can_packet = struct.pack(can_frame_format, can_id, can_dlc, kph_Tx)

    #print(can_packet)
    
    #Send out the CAN frame
    sock.send(can_packet)
    
    pass


## Cruise Control Module

This is the core of this pair of Jupyter notebooks. The following cell is a python function which takes in vehicle speed as a floating-point integer and passes out APP also as a floating-point integer. It applies a PI controller based on vehicle speed.

It works by calculating the speed error as the difference between current speed and target speed and using those to calculate proportional and integral terms. These two terms are simply added together to produce the APP output.

The proportional term is calculated by simply multiplying the current speed error by the proportional gain constant $\textit{Kp}$. 
The integral term is calculated by summing speed error between the current value and a fixed number of seconds into the past, or the $\textit{itc}$, and multiplying the result by the integral gain constant, $\textit{Ki}$.

$$ error = speed - target $$
$$ APP = Kp\cdot error + Ki\cdot \sum_{i-itc}^{i} error     $$

>where $\textit{i} $ is the current iteration and $\textit{itc}$ is the integrator time constant. 

After calculating an APP output, the CCM applies slew rate limits which limit how quickly the APP is allowed to change. This prevents abrupt changes to the APP, such as going from 20% to 100% within one 100ms loop. It is also best practice to include a slew rate in most industrial controllers. If APP is changing faster than the slew rate limit,  APP will be set to the highest allowable value. There are separate slew rate limits on both increasing APP and decreasing APP and are set by the user in the $\textit{User Inputs}$ cell above.



In [5]:
#Cruise Controller
def cruise_controller(kph):
    global i
    global prev_APP
    global APP
    global ki
    global kp

    error = kph - target
    prev_errors[i] = error
    i = i + 1
    if i == itc:
        i = 0
    kp = -c_kp * error
    ki = -c_ki * sum(prev_errors)
    APP = kp + ki

    if APP > 100:
        APP = 100
    elif APP < 0:
        APP = 0

    slew_rate = (APP - prev_APP) / t_interval

    if slew_rate < slew_lowLimit:
        slew_rate = slew_lowLimit
        APP = prev_APP + slew_rate * t_interval
    elif slew_rate > slew_highLimit:
        slew_rate = slew_highLimit
        APP = prev_APP + slew_rate * t_interval

    prev_APP = APP
    APP = APP
    return APP


## Main CCM Loop

The following cell is the main response loop for the cruise controller. It simply calls the CAN-receive, cruise controller, and CAN-transmit functions in a loop while checking for errors each loop.

In [6]:
# Main Response Loop: Cruise Control Module

while (checks == True):
    
    # Main Loop
    CAN_kph_Rx()
    cruise_controller(kph)
    CAN_APP_Tx(APP)
    
    # Error Handlers
    if abs(target - kph) > 30:
        print("ERROR: Target speed outside speed difference limit")
        checks = False
        break
      
    
    
# The following are not needed if running on a Beaglebone.
# These are used if running notebook on a PC that supports matplotlib and can graph the response

    # Generated Graph of Response
    #if graph == True:
    #    x_value+=t_interval
    #    y_value = kph
    #    g_value = grade
    #    x.append(x_value)
    #    y.append(y_value)
    #    gout.append(g_value)
    #    print(kph)
    #    print(APP)
    #    print(x_value)#Just put in to make sure program is running properly
    #    print(y_value) #Uncomment these three "print" statements to show numerical values
    #    # Use this stuff if running in a PC and can use matplotlib
    #    if x_value >= 10000: #Change this number for how long you want loop to run
    #        graph = False
    #        checks = False
    #        plt.plot(x,y)
    #        plt.plot(x,y2)
    #        plt.xlim(0,60)
    #        plt.ylim(0,110)
    #        plt.grid(True)
    #        plt.xlabel('Time [s]')
    #        plt.ylabel('Speed [kph]')
    #        plt.title('Cruise Control Response')
    #        plt.show()


OSError: [Errno 105] No buffer space available